In [1]:
# Useful starting lines
%matplotlib inline
from scipy.sparse import *
import numpy as np
import pickle
import random
import sys
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Data input and output paths
POS_TRAIN_PATH = 'data/twitter-datasets/train_pos_full.txt' 
NEG_TRAIN_PATH = 'data/twitter-datasets/train_neg_full.txt' 
DATA_TEST_PATH = 'data/twitter-datasets/test_data.txt'
OUTPUT_PATH = 'predictions_out.csv'

# For debugging purpose
np.set_printoptions(threshold= 50)
np.random.seed(10)

## Load the training data into feature matrix, class labels, and event ids:

In [5]:
''' DON'T RUN, YOU FOOL
!sh ./scripts/build_vocab.sh
!sh ./scripts/cut_vocab.sh
!python3 src/pickle_vocab.py
!python3 src/cooc.py
'''

" DON'T RUN, YOU FOOL\n!sh ./scripts/build_vocab.sh\n!sh ./scripts/cut_vocab.sh\n!python3 src/pickle_vocab.py\n!python3 src/cooc.py\n"

## Analyse Data


### Null values

In [7]:
''' DON'T RUN, YOU FOOL
print("loading cooccurrence matrix")
#with open('cooc.pkl', 'rb') as f:
with open('saved_gen_files/cooc.pkl', 'rb') as f:
    cooc = pickle.load(f)
print("{} nonzero entries".format(cooc.nnz))

nmax = 100
print("using nmax =", nmax, ", cooc.max() =", cooc.max())

print("initializing embeddings")
embedding_dim = 20
xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
ys = np.random.normal(size=(cooc.shape[1], embedding_dim))
'''

' DON\'T RUN, YOU FOOL\nprint("loading cooccurrence matrix")\n#with open(\'cooc.pkl\', \'rb\') as f:\nwith open(\'saved_gen_files/cooc.pkl\', \'rb\') as f:\n    cooc = pickle.load(f)\nprint("{} nonzero entries".format(cooc.nnz))\n\nnmax = 100\nprint("using nmax =", nmax, ", cooc.max() =", cooc.max())\n\nprint("initializing embeddings")\nembedding_dim = 20\nxs = np.random.normal(size=(cooc.shape[0], embedding_dim))\nys = np.random.normal(size=(cooc.shape[1], embedding_dim))\n'

In [8]:
''' DON'T RUN, YOU FOOL
eta = 0.001
alpha = 3 / 4

epochs = 10

for epoch in range(epochs):
    print("epoch {}".format(epoch))
    for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
        logn = np.log(n)
        fn = min(1.0, (n / nmax) ** alpha)
        x, y = xs[ix, :], ys[jy, :]
        scale = 2 * eta * fn * (logn - np.dot(x, y))
        xs[ix, :] += scale * y
        ys[jy, :] += scale * x
        
np.save('embeddings', xs)
'''

' DON\'T RUN, YOU FOOL\neta = 0.001\nalpha = 3 / 4\n\nepochs = 10\n\nfor epoch in range(epochs):\n    print("epoch {}".format(epoch))\n    for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):\n        logn = np.log(n)\n        fn = min(1.0, (n / nmax) ** alpha)\n        x, y = xs[ix, :], ys[jy, :]\n        scale = 2 * eta * fn * (logn - np.dot(x, y))\n        xs[ix, :] += scale * y\n        ys[jy, :] += scale * x\n        \nnp.save(\'embeddings\', xs)\n'

### Prepare the Data

In [5]:
embeddings = np.load('saved_gen_files/embeddings.npy')

train_text_neg = open(NEG_TRAIN_PATH, 'r').readlines()
train_text_pos = open(POS_TRAIN_PATH, 'r').readlines()
# Construct the two arrays 
train_text = np.array(train_text_neg + train_text_pos)
train_y = np.concatenate([np.zeros(len(train_text_neg)), np.ones(len(train_text_pos))])
train_text.shape

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 7072: character maps to <undefined>

In [10]:
with open('vocab.pkl', 'rb') as f:
    voc = pickle.load(f)

def toAvgVec(t):
    
    _, K = embeddings.shape
    sum_vec = np.zeros((K))
    words = t.split()
    for word in words:
        index = voc.get(word)
        if index is not None:
            sum_vec += embeddings[index]
            
    return sum_vec/len(words)
# Create numerical feature matrix of tweets
train_data = np.zeros(len(train_text)*embeddings.shape[1]).reshape(len(train_text), 20)
for i in range(len(train_text)):
    train_data[i] = toAvgVec(train_text[i])

In [16]:
indices = np.arange(train_data.shape[0])
random.shuffle(indices)

indices
X_train = train_data[indices[:2400000]]
y_train = train_y[indices[:2400000]]

X_test = train_data[2400000:]
y_test = train_y[2400000:]

In [17]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)

clf.fit(X_train, y_train)
clf.predict(X_test)

accuracy_score(y_test, clf.predict(X_test))

0.87394

### Outliers 